# Hvordan fordele eiendeler med utgangspunkt i ønskelister?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import cvxopt

## Maksimal flyt med Edmonds Karp

In [14]:
users = ['A', 'B', 'C']
assets = ['a', 'b', 'c', 'd', 'e']

wishlists = [np.random.permutation(len(assets)) for u in users]
wishlists

[array([0, 4, 2, 1, 3]), array([1, 3, 4, 0, 2]), array([3, 4, 0, 1, 2])]

In [32]:
class Edge():
    name = ""
    flows = {}
    capacities = {}
    
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return self.name + ": " + str(self.edges)
    
user_nodes = [Node(u) for u in users]
asset_nodes = [Node(a) for a in assets]
s = Node("s")
t = Node("t")

In [35]:
for i in range(len(users)):
    user_nodes[i].edges = {asset_nodes[j]: [0, wishlists[i][j]] for j in range(len(assets))} | {s: [0, len(assets)]}
    
for i in range(len(assets)):
    asset_nodes[i].edges = {user_nodes[j]: [0, wishlists[j][i]) for j in range(len(users))} | {t: (0, len(assets))}
    
s.edges = {u_n: (0 len(assets) for u_n in user_nodes}
t.edges = {a_n: len(assets) for a_n in asset_nodes}

In [48]:
flows = {(u_n, a_n): 0 for u_n, a_n in itertools.product(user_nodes, asset_nodes)}\
        | {(s, u_n): 0 for u_n in user_nodes}\
        | {(a_n, t): 0 for a_n in asset_nodes}

In [51]:
edges = list(flows.keys())

In [ ]:
flow = 0
while True:
    queue = [s]
    visited = []
    while len(q) > 0:
        current = queue.pop()
        for edge in current.edges:
            

Etter litt refleksjon har jeg kommet på at løsninger på maksimal flyt kan gi fordelinger der enkelteiendeler må deles mellom brukere. Dette er upraktisk i praksis. Vi kan derfor ikke bruke maksimal flyt.

## 0-1 Integer Lineær programmering
"Minimize c*x subject to Ax <=b, x ∈ {0, 1}"

In [205]:
asset_ids = ['a', 'b', 'c', 'd', 'e']
wishlist = ['c', 'd', 'a', 'b', 'e']
[asset_ids.index(asset_id) for asset_id in wishlist]

[2, 3, 0, 1, 4]

In [162]:
users = ['A', 'B', 'C']
assets = ['a', 'b', 'c', 'd', 'e']

wishlists = [np.random.permutation(len(assets)) for u in users]
c = np.hstack(wishlists) - len(assets)
c

array([-1, -5, -2, -3, -4, -3, -1, -2, -5, -4, -1, -4, -3, -5, -2])

In [163]:
wishlists

[array([4, 0, 3, 2, 1]), array([2, 4, 3, 0, 1]), array([4, 1, 2, 0, 3])]

In [164]:
[[1], [2]] + [[2], [3]]

[[1], [2], [2], [3]]

In [176]:
max_one_user_per_asset = [[1 if (i-j)%len(assets)==0 else 0 for i in range(len(c))] for j in range(len(assets))]
fair_distribution = [[c[i] if len(assets)*j <= i < len(assets)*(j+1) else 0 for i in range(len(c))] for j in range(len(users))]
A = np.array(max_one_user_per_asset + fair_distribution)
A

array([[ 1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1],
       [-1, -5, -2, -3, -4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, -3, -1, -2, -5, -4,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -4, -3, -5, -2]])

In [188]:
max_one_user_per_asset_sum = [1] * len(assets)
fair_distribution_sum = [c[:len(assets)].sum() / len(users)] * len(users)
b = np.array(max_one_user_per_asset_sum + fair_distribution_sum)
b

array([ 1.,  1.,  1.,  1.,  1., -5., -5., -5.])

In [189]:
A.shape

(8, 15)

In [190]:
G = cvxopt.matrix(A, tc='d')
G

<8x15 matrix, tc='d'>

In [191]:
h=cvxopt.matrix(b, tc='d')
h

<8x1 matrix, tc='d'>

In [192]:
from cvxopt.glpk import ilp

solution = ilp(c=cvxopt.matrix(c, tc='d'),
               G=cvxopt.matrix(A, tc='d'),
               h=cvxopt.matrix(b, tc='d'),
               B=set(range(len(c))))
solution

('optimal', <15x1 matrix, tc='d'>)

In [193]:
opt = np.array(list(solution[1]))
opt

array([0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0.])

In [194]:
np.matmul(A, opt)

array([ 1.,  1.,  1.,  1.,  1., -5., -7., -8.])

In [195]:
c

array([-1, -5, -2, -3, -4, -3, -1, -2, -5, -4, -1, -4, -3, -5, -2])

In [196]:
np.resize(np.multiply(c, opt), (len(users), len(assets)))

array([[-0., -5., -0., -0., -0.],
       [-3., -0., -0., -0., -4.],
       [-0., -0., -3., -5., -0.]])

In [197]:
# np.matmul(A, np.array(solution[1]))

Litt testing og logikk tilsier at denne algoritmen er optimal og rettferdig. Vi går derfor for binær linear programmering som løsning.

In [206]:
class C:
    pass

a = C()
b = C()

C.objects

AttributeError: type object 'C' has no attribute 'objects'